In [48]:
import pandas as pd
import numpy as np


import statsmodels.api as sm

from mlxtend.feature_selection import SequentialFeatureSelector as SFS
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import scale
from sklearn import preprocessing
from sklearn import metrics

file = pd.read_excel ('./Data/subset1.xlsx')


df = pd.DataFrame(file, columns=file.columns)

In [36]:
df.drop(df[pd.isnull(df['EU_Imgration'])].index, inplace=True)

In [37]:
df.drop('year', axis=1, inplace=True)
df.drop('id', axis=1, inplace=True)
df.drop('Population_growth_all', axis=1, inplace=True)
df.drop('Pop._Growth_rate', axis=1, inplace=True)
df.drop('Population_change', axis=1, inplace=True)
df.drop('Mortality_rate', axis=1, inplace=True)
df.drop('Fertility_Rates_%', axis=1, inplace=True)
df.drop('Births_Deaths', axis=1, inplace=True)


In [24]:
#df

In [38]:
X = df.drop('Population_growth',1 )

Y = df['Population_growth']


In [39]:
sfs = SFS(LinearRegression(),
         k_features=11,
          forward=True,
          floating=False,
          scoring = 'r2',
          cv = 0)

sfs.fit(X,Y)
sfs.k_feature_names_

('population',
 'No_Briths_E_W',
 'No_Births_NI',
 'No_Births_S',
 'No_births_male',
 'No_deaths_EW',
 'No_deaths_NI',
 'No_deaths_S',
 'Migration',
 'Migration_rate',
 'EU_Imgration')

In [40]:
x = pd.DataFrame(file, columns=[
 'population',
 'No_Briths_E_W',
 'No_Births_NI',
 'No_Births_S',
 'No_births_male',
 'No_deaths_EW',
 'No_deaths_NI',
 'No_deaths_S',
 'Migration',
 'Migration_rate',
 'EU_Imgration'])
y = pd.DataFrame(file, columns=['Population_growth'])


In [41]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

In [42]:
scaler = preprocessing.StandardScaler()

sx_train = scaler.fit_transform(x_train)
sx_test = scaler.fit_transform(x_test)

In [43]:
lr = LinearRegression()
lr.fit(sx_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [44]:
y_pred = lr.predict(sx_test)

In [45]:
#mean square error
np.sqrt(metrics.mean_squared_error(y_test, y_pred))

82784.4099950444

In [46]:
#compare predicted numbers with actual numbes 
check = np.column_stack((y_test, y_pred))
pd.DataFrame(check, columns=['actual', 'predicted'])

,actual,predicted
0,192565.0,234149.018757
1,37189.0,54108.740190
2,131070.0,146040.408264
3,470544.0,581516.310087
4,345585.0,532371.966542
5,121483.0,96032.353900
6,117125.0,125141.742976
7,376205.0,497532.365683
8,64995.0,103817.393147
9,137864.0,174042.204478
